In [2]:
import sys
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
# import preprocessing.preprocess
!pwd
# from legendary_game_recs.preprocessing import preprocess
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess import *

/Users/antonis/code/Ant-mel/legendary_game_recs/notebooks


In [3]:
raw_data = pd.read_csv('../raw_data/all_data_batch1_30to60k')
raw_2 = pd.read_csv('../raw_data/all_data_batch1_0to4k')
raw_3 = pd.read_csv('../raw_data/all_data_batch2_4kto8k')
raw_4 = pd.read_csv('../raw_data/all_data_batch3_8kto12k')
raw_5 = pd.read_csv('../raw_data/all_data_batch_60kto70k')
raw_6 = pd.read_csv('../raw_data/all_data_batch_70kto80k')
raw_7 = pd.read_csv('../raw_data/all_data_batch_80kto90k')
raw_8 = pd.read_csv('../raw_data/all_data_batch_90kto120k')

In [4]:
data = pd.concat((raw_2, raw_3, raw_4, raw_data, raw_5, raw_6, raw_7, raw_8), ignore_index=True)

In [5]:
def remove_generic_company_abbreviations(list_):
    """This removes trailing company abberviations that were apperaing as individual companies"""
    words_list = ['Ltd.', 'Inc.', 'LLC'] 
    
    for word in words_list:
        if word in list_:
            list_.remove(word)
    
    return list_

def keep_x_OHE_columns(OHE_coulmns, num_features=10):
    """Slices by the X (num_featutes) most common columns"""
    """Returns only the amount specified"""
    cols_by_power = pd.DataFrame(OHE_coulmns.sum().sort_values(ascending=False))
    
    top_20_OHE = list(cols_by_power[0:num_features].index)
    cols_to_keep = OHE_coulmns[top_20_OHE]
    
    return cols_to_keep

In [8]:
def keeping_ohe_columns_and_dropping_the_originals(df_of_columns, num_ohe):
    empty = pd.DataFrame()
    
    columns = list(df_of_columns.columns)
    
    for col in columns:
        encoded_col, col_encoder = categorical_encoder(df_of_columns[col])
        cols_to_keep = keep_x_OHE_columns(encoded_col, num_ohe)
        empty = pd.concat((empty, cols_to_keep), axis=1)
        
    return empty

In [9]:
categories_as_lists = make_list_columns_to_lists(data, ['developers', 'platforms', 'genres'])

In [14]:
df, enc = categorical_encoder(categories_as_lists['developers'])

In [16]:
keep_x_OHE_columns(df)

,,Nintendo,Sega,Konami,Electronic Arts,Inc.,Capcom,Activision,Sony Computer Entertainment,Square Enix
0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
101837,1,0,0,0,0,0,0,0,0,0
101838,1,0,0,0,0,0,0,0,0,0
101839,1,0,0,0,0,0,0,0,0,0
101840,1,0,0,0,0,0,0,0,0,0


In [10]:
ohe_columns = keeping_ohe_columns_and_dropping_the_originals(categories_as_lists, 10)

In [11]:
ohe_columns

,,Nintendo,Sega,Konami,Electronic Arts,Inc.,Capcom,Activision,Sony Computer Entertainment,Square Enix,...,Adventure,Indie,RPG,Simulator,Strategy,Shooter,Puzzle,,Arcade,Platform
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101837,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
101838,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
101839,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
101840,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [65]:
data_dropped_categories = data.drop(categories_as_lists.columns, axis=1)

In [77]:
data_encoded = data_dropped_categories.merge(ohe_columns, left_index=True, right_index=True)

# Developers

In [37]:
dev = categories_as_lists['developers'].apply(remove_generic_company_abbreviations)

encoded_dev, dev_encoder = categorical_encoder(dev)


In [56]:
devs_to_keep = keep_x_OHE_columns(encoded_dev, 10)

In [57]:
devs_to_keep

,,Nintendo,Sega,Konami,Electronic Arts,Capcom,Activision,Sony Computer Entertainment,Square Enix,THQ
0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
101837,1,0,0,0,0,0,0,0,0,0
101838,1,0,0,0,0,0,0,0,0,0
101839,1,0,0,0,0,0,0,0,0,0
101840,1,0,0,0,0,0,0,0,0,0


# Genres

In [50]:
genres = categories_as_lists['genres']

In [51]:
encoded_genre, genre_encoder = categorical_encoder(genres)

In [52]:
genre_by_power = encoded_genre.sum().sort_values(ascending=False)

In [59]:
genre_by_power

Adventure              30157
Indie                  27061
RPG                    14331
Simulator              13716
Strategy               13430
Shooter                11852
Puzzle                 11547
                        9993
Arcade                  9633
Platform                9443
Sport                   7541
Racing                  4941
Visual Novel            4688
Fighting                3710
Point-and-Click         2774
Brawler                 2718
Turn Based Strategy     2250
Music                   1830
Real Time Strategy      1715
Tactical                1669
Card & Board Game       1456
Quiz/Trivia             1011
Pinball                  406
MOBA                      52
dtype: int64

In [58]:
genre_to_keep = keep_x_OHE_columns(encoded_genre, 10)
genre_to_keep

,Adventure,Indie,RPG,Simulator,Strategy,Shooter,Puzzle,,Arcade,Platform
0,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
101837,0,0,0,0,0,0,0,1,0,0
101838,0,0,0,0,0,0,0,1,0,0
101839,0,0,0,0,0,0,0,1,0,0
101840,0,0,0,0,0,0,0,1,0,0


# Platforms

In [54]:
platforms = categories_as_lists['platforms']

encoded_plat, plat_encoder = categorical_encoder(platforms)

platforms_by_power = pd.DataFrame(encoded_plat.sum().sort_values(ascending=False))

In [60]:
platforms_to_keep = keep_x_OHE_columns(encoded_plat, 10)
platforms_to_keep

,Windows PC,Mac,PlayStation 4,Nintendo Switch,Linux,iOS,Xbox One,Android,PlayStation 3,Web browser
0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
101837,0,0,0,0,0,0,0,0,0,0
101838,0,0,0,0,0,0,0,0,0,0
101839,0,0,0,0,0,0,0,0,0,0
101840,0,0,0,0,0,0,0,0,0,0


In [ ]:
def make_stringlist_list(string):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    list_of_strings = string[2:-2].replace("'", '').split(',')
    return list_of_strings

def remove_whitespace(list_):
    '''This removes whitesapces from items within lists'''
    empty = []
    
    for i in range(len(list_)):
        item = list_[i].strip()
        empty.append(item)
    return empty

def clean_stringlists(df):
    '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
    string_to_list_df = df.apply(make_stringlist_list)
    remove_whitespace_df = string_to_list_df.apply(remove_whitespace)
    
    return remove_whitespace_df

In [ ]:
def make_list_columns_to_lists(df, columns):
    '''This returns a dataframe of columns where lists where imported as strings, and returns them to their list state'''
    cleaned_df = pd.DataFrame()
    
    for col in columns:
        cleaned = clean_stringlists(df[col])
        cleaned_df[col] = cleaned
    return cleaned_df

In [ ]:
updated_df = make_list_columns_to_lists(data, ['genres', 'platforms', 'developers'])

In [ ]:
updated_df

In [ ]:
mlb_genre = MultiLabelBinarizer()
mlb_platform = MultiLabelBinarizer()
mlb_developer = MultiLabelBinarizer()


transfomed_genre = mlb_genre.fit_transform(updated_df['genres'])
transformed_platform = mlb_platform.fit_transform(updated_df['platform'])
transfomed_developer = mlb_developer.fit_transform(updated_df['developer'])

In [ ]:
genre_encoded = pd.DataFrame(transfomed, columns=mlb.classes_)

In [ ]:
def categorical_encoder(category):
    mlb = MultiLabelBinarizer()
    transformed = mlb.fit_transform(category)
    
    df = pd.DataFrame(transformed, columns=mlb.classes_)
    
    return df, mlb


In [ ]:
df, mlb_thing = categorical_encoder(updated_df['developers'])

In [ ]:
df.shape

In [ ]:
genre_encoded

In [ ]:
def make_stringlist_list(string):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    list_of_strings = string[2:-2].replace("'", '').split(',')
    return list_of_strings

def remove_whitespace(list_):
    '''This removes whitesapces from items within lists'''
    empty = []

    for i in range(len(list_)):
        item = list_[i].strip()
        empty.append(item)
    return empty

def clean_stringlists(df):
    '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
    string_to_list_df = df.apply(make_stringlist_list)
    remove_whitespace_df = string_to_list_df.apply(remove_whitespace)

    return remove_whitespace_df

# This is the final one mentioned above
def make_list_columns_to_lists(df, columns):
    '''This returns a dataframe of columns where lists where imported as strings, and returns them to their list state'''
    cleaned_df = pd.DataFrame()

    for col in columns:
        cleaned = clean_stringlists(df[col])
        cleaned_df[col] = cleaned
    return cleaned_df